### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy import stats

from tqdm.autonotebook import tqdm, trange

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('../ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('../ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movie_info[movie_info['name'].apply(lambda x: 'Batman' in x)]

,movie_id,name,category
151,153,Batman Forever (1995),Action|Adventure|Comedy|Crime
588,592,Batman (1989),Action|Adventure|Crime|Drama
1356,1377,Batman Returns (1992),Action|Adventure|Comedy|Crime
1522,1562,Batman & Robin (1997),Action|Adventure|Crime
3144,3213,Batman: Mask of the Phantasm (1993),Animation|Children's


In [12]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [13]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '1526    Hercules (1997)',
 '2252    Pleasantville (1998)',
 '2692    Iron Giant, The (1999)',
 '1838    Mulan (1998)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [14]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [15]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [16]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [17]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1884    French Connection, The (1971)',
 '3458    Predator (1987)',
 '847    Godfather, The (1972)',
 '453    Fugitive, The (1993)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [18]:
import abc
from sklearn.metrics.pairwise import cosine_similarity

class Recommender(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def fit(self, user_items):
        raise NotImplementedError()
    
    def similar_items(self, item_id, k=10):
        assert self.items_embeddings is not None
        similarity = cosine_similarity(self.items_embeddings.T)
        return np.flip(np.argsort(similarity[item_id, :])[-k:]).reshape(-1, 1)
    
    def recommend(self, user_id, user_item_csr, k=10, exclude_viewed=True): 
        # чуть позже сделаю
        raise NotImplementedError

In [19]:
N_FACTORS = 64

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [20]:
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle

In [21]:
class SVD(Recommender):
    def __init__(self, n_factors, alpha, lr, n_iters, eps=1e-3):
        self.n_factors = n_factors
        self.alpha = alpha
        self.lr = lr
        self.n_iters = n_iters
    
    def fit(self, user_items):
        users_shape, items_shape = user_items.shape
        W = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(users_shape, self.n_factors))
        H = np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(self.n_factors, items_shape))

        W_bias = np.zeros(users_shape) #np.random.normal(loc=3, scale=2, size=users_shape)
        H_bias = np.zeros(items_shape) #np.random.normal(loc=3, scale=2, size=items_shape)
        global_bias = user_item.sum() / user_item.count_nonzero()

        pbar = trange(self.n_iters)
        for i in pbar:
            # shuffle train dataset
            rows, columns, data = shuffle(user_items.row, user_items.col, user_items.data)
            for i, j, v in zip(tqdm(rows, leave=False), columns, data):
                error = (W[i, :] @ H[:, j] + W_bias[i] + H_bias[j] + global_bias) - v
                old_w_row = W[i, :].copy()
                W_bias[i] -= self.lr * (error + self.alpha * W_bias[i])
                W[i, :] -= self.lr * (error * H[:, j].T + self.alpha * W[i, :])
                H_bias[j] -= self.lr * (error + self.alpha * H_bias[j])
                H[:, j] -= self.lr * (error * old_w_row.T + self.alpha * H[:, j])
    #             global_bias -= self.lr * (error + self.alpha * global_bias)

            loss = mean_squared_error(data, (W @ H + W_bias.reshape(-1, 1) + H_bias.reshape(1, -1) + global_bias)[rows, columns])
            print(f"mse loss: {loss}")
            pbar.set_postfix(loss=loss)
        
        self.user_embeddings = W
        self.items_embeddings = H
        return self

In [22]:
explicit_user_item = sp.coo_matrix((ratings["rating"], (ratings["user_id"], ratings["movie_id"])))

In [23]:
svd = SVD(n_factors=N_FACTORS, alpha=1e-2, lr=0.01, n_iters=10).fit(explicit_user_item)

mse loss: 0.8544045643762032


mse loss: 0.8200417861694577


mse loss: 0.8016559186768909


mse loss: 0.7737024882608337


mse loss: 0.7392965044133978


mse loss: 0.7006930479132008


mse loss: 0.6607658347891784


mse loss: 0.6206918509620251


mse loss: 0.5813739533894536


mse loss: 0.5446724741335722



In [24]:
get_similars(1, svd)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)',
 '1526    Hercules (1997)',
 '1838    Mulan (1998)',
 '3175    Goodbye Girl, The (1977)',
 '3682    Chicken Run (2000)',
 '2021    Rescuers, The (1977)']

In [25]:
get_similars(2628, svd)

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1840    X-Files: Fight the Future, The (1998)',
 '1954    Godfather: Part III, The (1990)',
 '313    Stargate (1994)',
 '1180    Raiders of the Lost Ark (1981)',
 '1014    Robin Hood: Prince of Thieves (1991)']

**в топе хорошие рекомендации: мультфильмы и другие части звездных войн**

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [26]:
import scipy
from scipy import sparse
from scipy.sparse.linalg import spsolve

In [27]:
class ALS(Recommender):
    def __init__(self, alpha, reg, n_factors, n_iters, eps=1e-3):
        self.alpha = alpha
        self.reg = reg
        self.n_factors = n_factors
        self.n_iters = n_iters
        self.eps = eps
        
    def __als_step(self, C, R, X, Y):
        # чтобы не считать на каждой итерации цикла
        YtY = Y.T.dot(Y)
        Y_eye = scipy.sparse.eye(Y.shape[0])
        lambda_eye = self.reg * scipy.sparse.eye(self.n_factors)

        for i in trange(X.shape[0], leave=False):
            # выбираем конкретный пример
            conf_samp = C[i, :].toarray()
            r = R[i, :].toarray()
            
            # уравнение 4 отсюда http://yifanhu.net/PUB/cf.pdf и фикс bottleneck
            cui_loc = scipy.sparse.diags(conf_samp, [0])
            A = YtY + Y.T @ cui_loc @ Y + lambda_eye
            b = Y.T @ (cui_loc + Y_eye) @ r.T 

            X[i] = spsolve(A, b)

        return X
    
    def fit(self, user_items):
        confidence  = self.alpha * user_items
        
        users_shape, items_shape = user_items.shape
        X = sparse.csr_matrix(np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(users_shape, self.n_factors)))
        Y = sparse.csr_matrix(np.random.uniform(0, 1 / np.sqrt(self.n_factors), size=(items_shape, self.n_factors))) 

        for iter_step in trange(self.n_iters): 
            X = self.__als_step(confidence, user_items, X, Y)
            Y = self.__als_step(confidence.T, user_items.T, Y, X)
            
        self.user_embeddings = X
        self.items_embeddings = Y.T
        
        return self

In [28]:
als = ALS(alpha=40, reg=0.1, n_factors=N_FACTORS, n_iters=5).fit(user_item_csr)

In [29]:
get_similars(1, als)

['0    Toy Story (1995)',
 '1245    Groundhog Day (1993)',
 '3045    Toy Story 2 (1999)',
 '2327    Shakespeare in Love (1998)',
 '33    Babe (1995)',
 '2693    Sixth Sense, The (1999)',
 '315    Shawshank Redemption, The (1994)',
 '589    Silence of the Lambs, The (1991)',
 '352    Forrest Gump (1994)',
 "2286    Bug's Life, A (1998)"]

In [30]:
get_similars(2628, als)

['2559    Star Wars: Episode I - The Phantom Menace (1999)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '1539    Men in Black (1997)',
 '476    Jurassic Park (1993)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '2502    Matrix, The (1999)',
 '585    Terminator 2: Judgment Day (1991)',
 '1959    Saving Private Ryan (1998)',
 '108    Braveheart (1995)']

**тоже выглядит неплохо, однако в топе популярное, возможно, bias'ы подправят это**

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных